# Account - Owner Services PROD

## Database Summary

In [7]:
Import-Module AWS.Tools.Common
Import-Module AWS.Tools.DynamoDBv2
Import-Module AWS.Tools.RDS

$Profiles   = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match 'DevOps-Owner-Services-PROD' }
$Regions    = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"

function Get-AWSDBInformation {
    [CmdletBinding()]
    param (

        $Profiles,
        $Regions,
        $RegexGroup             = 'AccountNumber',
        $RegexGroupAccountName  = 'AccountName',
        $RegexAccountNumber     = "(?<$RegexGroup>\d+):db.+$",
        $RegexAccountNumberDDb  = "(?<$RegexGroup>\d+):table.+$", 
        $RegexAccountName       = "(?<PreFix>DevOps-)(?<$RegexGroupAccountName>.+)"
    )
    
    begin {

        $Summary    = @()
        
    }
    
    process {

        $Summary    = @()
        foreach ($Profile in $Profiles) {

            foreach ($Region in $Regions) {
                
                $RDSInstances   = Get-RDSDBInstance -ProfileName $Profile.ProfileName -Region $Region
                $ddbTables      = Get-DDBTableList -ProfileName $Profile.ProfileName -Region $Region

                switch ([string]::IsNullOrEmpty($RDSInstances)) {
                    $true { 
                        if ($ddbTables) {
                            
                            $Table = $ddbTables | Select-Object -First 1
                            $ddbTable = Get-DDBTable -TableName $Table -ProfileName $Profile.ProfileName -Region $Region
                            $AccountID  = (($ddbTable.TableArn | Select-String -Pattern $RegexAccountNumberDDb).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                        }else {

                            $AccountID = 0
                        }
                    }
                    $false {

                        $RDSInstance   = $RDSInstances | Select-Object -First 1
                        $AccountID  = (($RDSInstance.DBInstanceArn | Select-String -Pattern $RegexAccountNumber).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                    }
                    default {

                        $AccountID = 0
                    }
                }

                $Sum = [PSCustomObject]@{

                    Account             = (($Profile.ProfileName | select-string -Pattern $RegexAccountName).Matches.Groups | Where-Object { $_.Name -eq $RegexGroupAccountName }).Value
                    AccountID           = $AccountID
                    Region              = $Region
                    RDSInstances        = ($RDSInstances.DBInstanceIdentifier | Measure-Object).Count
                    DynamoDBTables      = $ddbTables.Count
                    
                }

                $condition = $Sum.RDSInstances + $Sum.DynamoDBTables
                if ($condition -gt 0) {
                    $Summary += $Sum
                }
            }

        }
        
    }
    
    end {

        $Summary
        
    }
}

$Summary    = Get-AWSDBInformation $Profiles $Regions
$Summary    | Format-Table -AutoSize


Account             AccountID    Region    RDSInstances DynamoDBTables
-------             ---------    ------    ------------ --------------
Owner-Services-PROD 496175963148 eu-west-2            3              1



### DynamoDB Tables

In [8]:
$ddbBackupList  = Get-DDBBackupList -ProfileName $Profiles.ProfileName
$ddbTableList   = Get-DDBTableList -ProfileName $Profiles.ProfileName

$ddbTables  = @()
foreach ($ddbTable in $ddbTableList) {
    
    $Table = Get-DDBTable -TableName $ddbTable -ProfileName $Profiles.ProfileName

    if ($ddbBackupList) {
        
        $BackupAvailable = $ddbBackupList.TableName.Contains("$($Table.TableName)")
    }
    else {

        $BackupAvailable = $false
    }
    

    $Sum = [PSCustomObject]@{

        TableName           = $Table.TableName
        TableBackup         = $BackupAvailable
        CreationDateTime    = $Table.CreationDateTime
        ItemCount           = $Table.ItemCount
        # TableSizeBytes      = $Table.TableSizeBytes
        TableSizeMB         = "$([math]::round($($Table.TableSizeBytes /1MB), 0)) MB"
        TableStatus         = $Table.TableStatus
        # TableId             = $Table.TableId
        # KeySchema           = $Table.KeySchema
        # GlobalSecondaryIndexes = $Table.GlobalSecondaryIndexes
        
    }
    $ddbTables += $Sum
}
$ddbTables | Format-Table -AutoSize


TableName       TableBackup CreationDateTime      ItemCount TableSizeMB TableStatus
---------       ----------- ----------------      --------- ----------- -----------
terraform-locks       False 12/12/2022 1:05:57 PM        70 0 MB        ACTIVE



### RDS Instances

In [9]:
$RDSInstances = Get-RDSDBInstance -ProfileName $Profiles.ProfileName -Region eu-west-2

$Inst = @()
$Snap = @()
$Snapshots = @()
foreach ($Instance in $RDSInstances) {
    
    $DBInstanceArn      = $Instance.DBInstanceArn
    # $Tags               = Get-RDSTagForResource -ResourceName $DBInstanceArn -ProfileName $Profile.ProfileName
    $Shots          = Get-RDSDBSnapshot -ProfileName $Profiles.ProfileName -DBInstanceIdentifier $DBInstanceArn
    #$DBClusterSnapshots = Get-RDSDBClusterSnapshot -ProfileName $Profile.ProfileName -DBClusterIdentifier $DBInstanceArn

    $Sum = [PSCustomObject]@{

        #DBName                  = $Instance.DBName
        DBInstanceIdentifier    = $Instance.DBInstanceIdentifier
        Engine                  = $Instance.Engine
        EngineVersion           = $Instance.EngineVersion
        InstanceCreateTime      = $Instance.InstanceCreateTime
        #DBInstanceClass         = $Instance.DBInstanceClass
        BackupRetention         = $Instance.BackupRetentionPeriod
        #StorageThroughput       = $Instance.StorageThroughput
        #Tags                    = $Tags
        Snapshots               = ($Shots | Measure-Object).Count
        #DBClusterSnapshots      = ($DBClusterSnapshots | Measure-Object).Count
        
    }
    $Inst += $Sum
    $Snapshots += $Shots
    
}

$Snapshots | ForEach-Object {

    $Sum = [PSCustomObject]@{

        DBInstanceIdentifier    = $_.DBInstanceIdentifier
        DBSnapshotIdentifier    = $_.DBSnapshotIdentifier
        SnapshotCreateTime      = $_.SnapshotCreateTime
        Engine                  = $_.Engine
        SnapshotType            = $_.SnapshotType
        Status                  = $_.Status
        
    }
    $Snap += $Sum
}
$Inst | Format-Table -AutoSize


DBInstanceIdentifier Engine   EngineVersion InstanceCreateTime   BackupRetention Snapshots
-------------------- ------   ------------- ------------------   --------------- ---------
authenticationdbprod postgres 16.1          3/12/2024 8:57:35 AM               7        10
bookingdbprod        postgres 16.1          3/12/2024 9:02:23 AM               7         9
notificationdbprod   postgres 16.3          3/12/2024 8:57:15 AM               7        10



#### Snapshots

##### authenticationdbprod

In [10]:
$DB = 'authenticationdbprod'
$Snap | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object DBSnapshotIdentifier, SnapshotCreateTime, SnapshotType, Status |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


DBSnapshotIdentifier                      SnapshotCreateTime   SnapshotType Status
--------------------                      ------------------   ------------ ------
rds:authenticationdbprod-2024-07-12-02-57 7/12/2024 3:57:18 AM automated    available
rds:authenticationdbprod-2024-07-11-02-56 7/11/2024 3:56:46 AM automated    available
rds:authenticationdbprod-2024-07-10-02-56 7/10/2024 3:56:55 AM automated    available
rds:authenticationdbprod-2024-07-09-02-56 7/9/2024 3:56:51 AM  automated    available
rds:authenticationdbprod-2024-07-08-02-56 7/8/2024 3:56:40 AM  automated    available
rds:authenticationdbprod-2024-07-07-02-56 7/7/2024 3:56:51 AM  automated    available
rds:authenticationdbprod-2024-07-06-02-56 7/6/2024 3:56:41 AM  automated    available
rds:authenticationdbprod-2024-07-05-02-56 7/5/2024 3:56:40 AM  automated    available
rds:authenticationdbprod-2024-07-04-02-56 7/4/2024 3:56:27 AM  automated    available
authenticationdbprod-snapshot             3/12/2024 8:27:02

##### bookingdbprod

In [11]:
$DB = 'bookingdbprod'
$Snap | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object DBSnapshotIdentifier, SnapshotCreateTime, SnapshotType, Status |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


DBSnapshotIdentifier               SnapshotCreateTime   SnapshotType Status
--------------------               ------------------   ------------ ------
rds:bookingdbprod-2024-07-12-04-56 7/12/2024 5:56:45 AM automated    available
rds:bookingdbprod-2024-07-11-04-56 7/11/2024 5:56:58 AM automated    available
rds:bookingdbprod-2024-07-10-04-56 7/10/2024 5:56:56 AM automated    available
rds:bookingdbprod-2024-07-09-04-56 7/9/2024 5:56:49 AM  automated    available
rds:bookingdbprod-2024-07-08-04-56 7/8/2024 5:56:57 AM  automated    available
rds:bookingdbprod-2024-07-07-04-56 7/7/2024 5:56:59 AM  automated    available
rds:bookingdbprod-2024-07-06-04-56 7/6/2024 5:56:30 AM  automated    available
rds:bookingdbprod-2024-07-05-04-57 7/5/2024 5:57:14 AM  automated    available
bookingdbprod-snapshot             3/12/2024 8:38:57 AM manual       available



##### notificationdbprod

In [12]:
$DB = 'notificationdbprod'
$Snap | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object DBSnapshotIdentifier, SnapshotCreateTime, SnapshotType, Status |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


DBSnapshotIdentifier                    SnapshotCreateTime   SnapshotType Status
--------------------                    ------------------   ------------ ------
rds:notificationdbprod-2024-07-12-00-20 7/12/2024 1:20:25 AM automated    available
rds:notificationdbprod-2024-07-11-00-20 7/11/2024 1:20:29 AM automated    available
rds:notificationdbprod-2024-07-10-00-20 7/10/2024 1:20:46 AM automated    available
rds:notificationdbprod-2024-07-09-00-20 7/9/2024 1:20:43 AM  automated    available
rds:notificationdbprod-2024-07-08-00-21 7/8/2024 1:21:16 AM  automated    available
rds:notificationdbprod-2024-07-07-00-20 7/7/2024 1:20:33 AM  automated    available
rds:notificationdbprod-2024-07-06-00-20 7/6/2024 1:20:23 AM  automated    available
rds:notificationdbprod-2024-07-05-00-20 7/5/2024 1:20:51 AM  automated    available
rds:notificationdbprod-2024-07-04-00-20 7/4/2024 1:20:16 AM  automated    available
notificationdbprod-snapshot             3/12/2024 8:39:20 AM manual       availab